# In-Memory DB

## Level 1

In [16]:
import typing as tp


def solution(queries) -> tp.List[str]:
    db_ = {}

    def set(key, field, value) -> str:
        if key not in db_:
            db_[key] = {field: value}
        else:
            db_[key].update({field: value})
        return ""

    def get(key, field) -> str:
        return db_.get(key, {}).get(field, "")

    def delete(key, field) -> str:
        if key not in db_:
            return "false"

        if field not in db_[key]:
            return "false"

        db_[key].pop(field)
        return "true"

    result = []
    for q in queries:
        if q[0] == "SET":
            result.append(set(q[1], q[2], q[3]))
        elif q[0] == "GET":
            result.append(get(q[1], q[2]))
        elif q[0] == "DELETE":
            result.append(delete(q[1], q[2]))

        # print(db_)

    return result


queries = [
    ["SET", "A", "B", "E"],
    ["SET", "A", "C", "F"],
    ["GET", "A", "B"],
    ["GET", "A", "D"],
    ["DELETE", "A", "B"],
    ["DELETE", "A", "D"],
]
result = solution(queries)
print(result)

['', '', 'E', '', 'true', 'false']


## Level 2

In [21]:
import typing as tp


def solution(queries) -> tp.List[str]:
    db_ = {}

    def set(key, field, value) -> str:
        if key not in db_:
            db_[key] = {field: value}
        else:
            db_[key].update({field: value})
        return ""

    def get(key, field) -> str:
        return db_.get(key, {}).get(field, "")

    def delete(key, field) -> str:
        if key not in db_:
            return "false"

        if field not in db_[key]:
            return "false"

        db_[key].pop(field)
        return "true"

    def scan(key) -> str:
        if key not in db_:
            return ""

        if not db_[key]:
            return ""

        records = []
        for field, value in sorted(db_[key].items()):
            records.append(f"{field}({value})")

        return ", ".join(records)

    def scan_by_prefix(key, prefix) -> str:
        if key not in db_:
            return ""

        if not db_[key]:
            return ""

        records = []
        for field, value in sorted(db_[key].items()):
            if field.startswith(prefix):
                records.append(f"{field}({value})")

        return ", ".join(records)

    result = []
    for q in queries:
        if q[0] == "SET":
            result.append(set(q[1], q[2], q[3]))
        elif q[0] == "GET":
            result.append(get(q[1], q[2]))
        elif q[0] == "DELETE":
            result.append(delete(q[1], q[2]))
        elif q[0] == "SCAN":
            result.append(scan(q[1]))
        elif q[0] == "SCAN_BY_PREFIX":
            result.append(scan_by_prefix(q[1], q[2]))

        # print(db_)

    return result


queries = [
    ["SET", "A", "C", "G"],
    ["SET", "A", "BD", "F"],
    ["SET", "A", "BC", "E"],
    ["SCAN_BY_PREFIX", "A", "B"],
    ["SCAN", "A"],
    ["SCAN_BY_PREFIX", "B", "B"],
]
result = solution(queries)
print(result)

['', '', '', 'BC(E), BD(F)', 'BC(E), BD(F), C(G)', '']


## Level 3

In [31]:
import typing as tp


def solution(queries) -> tp.List[str]:
    db_ = {}

    def set_at(key, field, value, timestamp):
        if key not in db_:
            db_[key] = {field: (value, float(timestamp), float("inf"))}
        else:
            db_[key].update({field: (value, float(timestamp), float("inf"))})
        return ""

    def set_at_with_ttl(key, field, value, timestamp, ttl):
        if key not in db_:
            db_[key] = {field: (value, float(timestamp), float(ttl))}
        else:
            db_[key].update({field: (value, float(timestamp), float(ttl))})
        return ""

    def delete_at(key, field, timestamp):
        if key not in db_:
            return "false"

        if field not in db_[key]:
            return "false"

        val_info = db_[key][field]
        if float(timestamp) >= val_info[1] + val_info[2]:
            return "false"

        db_[key].pop(field)
        return "true"

    def get_at(key, field, timestamp):
        if key not in db_:
            return ""

        if field not in db_[key]:
            return ""

        val_info = db_[key][field]
        if float(timestamp) >= val_info[1] + val_info[2]:
            return ""

        return val_info[0]

    def scan_at(key, timestamp):
        if key not in db_:
            return ""

        if not db_[key]:
            return ""

        records = []
        for field, val_info in sorted(db_[key].items()):
            if float(timestamp) < val_info[1] + val_info[2]:
                records.append(f"{field}({val_info[0]})")

        return ", ".join(records)

    def scan_by_prefix_at(key, prefix, timestamp):
        if key not in db_:
            return ""

        if not db_[key]:
            return ""

        records = []
        for field, val_info in sorted(db_[key].items()):
            if field.startswith(prefix) and float(timestamp) < val_info[1] + val_info[2]:
                records.append(f"{field}({val_info[0]})")

        return ", ".join(records)

    result = []
    for q in queries:
        if q[0] == "SET":
            # WARNING: backward compatibility
            result.append(set_at(q[1], q[2], q[3], timestamp=0))
        elif q[0] == "SET_AT":
            result.append(set_at(q[1], q[2], q[3], q[4]))
        elif q[0] == "SET_AT_WITH_TTL":
            result.append(set_at_with_ttl(q[1], q[2], q[3], q[4], q[5]))
        elif q[0] == "GET":
            result.append(get_at(q[1], q[2], timestamp=0))
        elif q[0] == "GET_AT":
            result.append(get_at(q[1], q[2], q[3]))
        elif q[0] == "DELETE":
            result.append(delete_at(q[1], q[2], timestamp=0))
        elif q[0] == "DELETE_AT":
            result.append(delete_at(q[1], q[2], q[3]))
        elif q[0] == "SCAN":
            result.append(scan_at(q[1], timestamp=0))
        elif q[0] == "SCAN_BY_PREFIX":
            result.append(scan_by_prefix_at(q[1], q[2], timestamp=0))
        elif q[0] == "SCAN_AT":
            result.append(scan_at(q[1], q[2]))
        elif q[0] == "SCAN_BY_PREFIX_AT":
            result.append(scan_by_prefix_at(q[1], q[2], q[3]))

        # print(db_)

    return result


queries = [
    # Backward compatibility
    # ["SET", "A", "C", "G"],
    # ["SET", "A", "BD", "F"],
    # ["SET", "A", "BC", "E"],
    # ["SCAN_BY_PREFIX", "A", "B"],
    # ["SCAN", "A"],
    # ["SCAN_BY_PREFIX", "B", "B"],
    # Scan with TTL
    # ["SET_AT_WITH_TTL", "A", "BC", "E", "1", "9"],
    # ["SET_AT_WITH_TTL", "A", "BC", "E", "5", "10"],
    # ["SET_AT", "A", "BD", "F", "5"],
    # ["SCAN_BY_PREFIX_AT", "A", "B", "14"],
    # ["SCAN_BY_PREFIX_AT", "A", "B", "15"],
    # Delete with TTL
    # ["SET_AT", "A", "B", "C", "1"],
    # ["SET_AT_WITH_TTL", "X", "Y", "Z", "2", "15"],
    # ["GET_AT", "X", "Y", "3"],
    # ["SET_AT_WITH_TTL", "A", "D", "E", "4", "10"],
    # ["SCAN_AT", "A", "13"],
    # ["SCAN_AT", "X", "16"],
    # ["SCAN_AT", "X", "17"],
    # ["DELETE_AT", "X", "Y", "20"],
]
result = solution(queries)
print(result)

['', '', 'C']


## Level 4

In [60]:
import typing as tp
import copy


def solution(queries) -> tp.List[str]:
    db_ = {}
    backup_ = []

    def set_at(key, field, value, timestamp):
        if key not in db_:
            db_[key] = {field: (value, float(timestamp), float("inf"))}
        else:
            db_[key].update({field: (value, float(timestamp), float("inf"))})
        return ""

    def set_at_with_ttl(key, field, value, timestamp, ttl):
        if key not in db_:
            db_[key] = {field: (value, float(timestamp), float(ttl))}
        else:
            db_[key].update({field: (value, float(timestamp), float(ttl))})
        return ""

    def delete_at(key, field, timestamp):
        if key not in db_:
            return "false"

        if field not in db_[key]:
            return "false"

        val_info = db_[key][field]
        if float(timestamp) >= val_info[1] + val_info[2]:
            return "false"

        db_[key].pop(field)
        return "true"

    def get_at(key, field, timestamp):
        if key not in db_:
            return ""

        if field not in db_[key]:
            return ""

        val_info = db_[key][field]
        if float(timestamp) >= val_info[1] + val_info[2]:
            return ""

        return val_info[0]

    def scan_at(key, timestamp):
        if key not in db_:
            return ""

        if not db_[key]:
            return ""

        records = []
        for field, val_info in sorted(db_[key].items()):
            if float(timestamp) < val_info[1] + val_info[2]:
                records.append(f"{field}({val_info[0]})")

        return ", ".join(records)

    def scan_by_prefix_at(key, prefix, timestamp) -> str:
        if key not in db_:
            return ""

        if not db_[key]:
            return ""

        records = []
        for field, val_info in sorted(db_[key].items()):
            if field.startswith(prefix) and float(timestamp) < val_info[1] + val_info[2]:
                records.append(f"{field}({val_info[0]})")

        return ", ".join(records)

    def backup(timestamp) -> str:
        # WARNING: use copy
        backup_.append((timestamp, copy.deepcopy(db_)))
        count = 0
        for key, field_info in db_.items():
            if not field_info:
                continue

            is_valid = False
            for field, val_info in field_info.items():
                if float(timestamp) < val_info[1] + val_info[2]:
                    is_valid = True
                    break

            if is_valid:
                count = count + 1

        return str(count)

    def restore(timestamp, timestampToRestore):
        # WARNING: use nonlocal to update the outscope variable
        nonlocal db_
        for pair in reversed(backup_):
            if pair[0] <= timestampToRestore:
                # WARNING: use copy
                db_ = copy.deepcopy(pair[1])
                break

        for key, field_info in db_.items():
            for field, val_info in field_info.items():
                remaining_ttl = val_info[2] - (float(pair[0]) - val_info[1])
                # WARNING: Update Python dict values
                db_[key].update({field: (val_info[0], float(timestamp), remaining_ttl)})
        return ""

    result = []
    for q in queries:
        if q[0] == "SET":
            result.append(set_at(q[1], q[2], q[3], timestamp=0))
        elif q[0] == "SET_AT":
            result.append(set_at(q[1], q[2], q[3], q[4]))
        elif q[0] == "SET_AT_WITH_TTL":
            result.append(set_at_with_ttl(q[1], q[2], q[3], q[4], q[5]))
        elif q[0] == "GET":
            result.append(get_at(q[1], q[2], timestamp=0))
        elif q[0] == "GET_AT":
            result.append(get_at(q[1], q[2], q[3]))
        elif q[0] == "DELETE":
            result.append(delete_at(q[1], q[2], timestamp=0))
        elif q[0] == "DELETE_AT":
            result.append(delete_at(q[1], q[2], q[3]))
        elif q[0] == "SCAN":
            result.append(scan_at(q[1], timestamp=0))
        elif q[0] == "SCAN_BY_PREFIX":
            result.append(scan_by_prefix_at(q[1], q[2], timestamp=0))
        elif q[0] == "SCAN_AT":
            result.append(scan_at(q[1], q[2]))
        elif q[0] == "SCAN_BY_PREFIX_AT":
            result.append(scan_by_prefix_at(q[1], q[2], q[3]))
        elif q[0] == "BACKUP":
            result.append(backup(q[1]))
        elif q[0] == "RESTORE":
            result.append(restore(q[1], q[2]))

        print(db_)

    return result


queries = [
    # Backward compatibility
    # ["SET", "A", "C", "G"],
    # ["SET", "A", "BD", "F"],
    # ["SET", "A", "BC", "E"],
    # ["SCAN_BY_PREFIX", "A", "B"],
    # ["SCAN", "A"],
    # ["SCAN_BY_PREFIX", "B", "B"],
    # Scan with TTL
    # ["SET_AT_WITH_TTL", "A", "BC", "E", "1", "9"],
    # ["SET_AT_WITH_TTL", "A", "BC", "E", "5", "10"],
    # ["SET_AT", "A", "BD", "F", "5"],
    # ["SCAN_BY_PREFIX_AT", "A", "B", "14"],
    # ["SCAN_BY_PREFIX_AT", "A", "B", "15"],
    # Delete with TTL
    # ["SET_AT", "A", "B", "C", "1"],
    # ["SET_AT_WITH_TTL", "X", "Y", "Z", "2", "15"],
    # ["GET_AT", "X", "Y", "3"],
    # ["SET_AT_WITH_TTL", "A", "D", "E", "4", "10"],
    # ["SCAN_AT", "A", "13"],
    # ["SCAN_AT", "X", "16"],
    # ["SCAN_AT", "X", "17"],
    # ["DELETE_AT", "X", "Y", "20"],
    # Empty records
    # ["SET_AT", "A", "B", "C", "1"],
    # ["SCAN_AT", "A", "2"],
    # ["DELETE_AT", "A", "B", "3"],
    # ["SCAN_AT", "A", "4"],
    # Backup
    # ["SET_AT_WITH_TTL", "A", "B", "C", "1", "10"],
    # ["BACKUP", "3"],
    # ["SET_AT", "A", "D", "E", "4"],
    # ["BACKUP", "5"],
    # ["DELETE_AT", "A", "B", "8"],
    # ["BACKUP", "9"],
    # ["RESTORE", "10", "7"],
    # ["BACKUP", "11"],
    # ["SCAN_AT", "A", "15"],
    # ["SCAN_AT", "A", "16"],
    #
    # ["BACKUP", "160000000"],
    # ["RESTORE", "160000001", "160000000"],
    # ["SET_AT_WITH_TTL", "key", "field", "str", "160000100", "200"],
    # ["BACKUP", "160000200"],
    # ["RESTORE", "160000250", "160000000"],
    # ["GET_AT", "key", "field", "160000300"],
    # ["BACKUP", "160000350"],
    # ["RESTORE", "160000400", "160000200"],
    # ["GET_AT", "key", "field", "160000450"],
    # ["GET_AT", "key", "field", "160000500"],
    #
    # ["SET_AT", "foo", "bar", "baz", "160000100"],
    # ["SET_AT_WITH_TTL", "key", "key", "value", "160000120", "1880"],
    # ["SET_AT_WITH_TTL", "key", "key", "value", "160000170", "680"],
    # ["SET_AT_WITH_TTL", "bar", "baz", "foo", "160000200", "100"],
    # ["BACKUP", "160000250"],
    # ["BACKUP", "160000270"],
    # ["SET_AT_WITH_TTL", "boo", "text1", "text2", "160000300", "900"],
    # ["BACKUP", "160000850"],
    # ["GET_AT", "foo", "bar", "160000900"],
    # ["SCAN_BY_PREFIX_AT", "key", "k", "160000920"],
    # ["DELETE_AT", "foo", "bar", "160000950"],
    # ["DELETE_AT", "key", "key", "160000960"],
    # ["RESTORE", "160000970", "160000250"],
    # ["SCAN_AT", "key", "160000980"],
    # ["SCAN_AT", "key", "160001021"],
    # ["RESTORE", "160001030", "160000850"],
    # ["SCAN_AT", "key", "160001040"],
    # ["SCAN_AT", "boo", "160001041"],
    # ["SCAN_AT", "bar", "160001042"],
    #
    # ["DELETE_AT", "key", "key", "160000010"],
    # ["DELETE_AT", "key", "key2", "160000020"],
    # ["SCAN_BY_PREFIX_AT", "key", "key", "160000025"],
    # ["GET_AT", "A", "B", "160000030"],
    # ["GET_AT", "key", "key", "160000040"],
    # ["SCAN_BY_PREFIX_AT", "key", "key", "160000050"],
    # ["DELETE_AT", "key", "key", "160000052"],
    # ["BACKUP", "160000055"],
    # ["SET_AT_WITH_TTL", "key", "key", "aaaaa", "160000060", "1940"],
    # ["SET_AT_WITH_TTL", "foo", "bar", "baz", "160000070", "101"],
    # ["DELETE_AT", "key", "bar", "160000080"],
    # ["DELETE_AT", "key", "key2", "160000090"],
    # ["BACKUP", "160000100"],
    # ["SET_AT_WITH_TTL", "key", "key", "otherValue", "160000120", "20"],
    # ["RESTORE", "160000130", "160000099"],
    # ["SCAN_BY_PREFIX_AT", "key", "key", "160000150"],
    # ["RESTORE", "160000160", "160000100"],
    # ["SCAN_BY_PREFIX_AT", "key", "k", "160000200"],
    # ["SCAN_BY_PREFIX_AT", "key", "k", "160000201"],
    # ["RESTORE", "160000250", "160000110"],
    # ["SCAN_AT", "key", "160000270"],
    # ["SCAN_BY_PREFIX_AT", "key", "key", "160000350"],
]
result = solution(queries)
print(result)

{'A': {'B': ('C', 1.0, inf)}}
{'A': {'B': ('C', 1.0, inf)}}
{'A': {}}
{'A': {}}
['', 'B(C)', 'true', '']
